In [1]:
# _September_Week7 2018: 9/01/18-12/24/18

import pandas as pd
import numpy as np
import datetime
import pandas as pd
from googleads import adwords
import google_analytics_reporting_api as ga
import os

week="_September_by_Day_by_DMA"

Start_Date_overall='2018-09-01'
End_Date_overall='2018-12-24'

today_str=str(datetime.datetime.now().date())
writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"+week[1:]+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

# Google Analytics

In [2]:
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].apply(lambda x: int(x.replace(",","")))
        df['ga:sessions']=df['ga:sessions'].astype(int)

    if 'ga:adClicks' in df.columns:
        df['ga:adClicks']=df['ga:adClicks'].apply(lambda x: int(x.replace(",","")))
        df['ga:adClicks']=df['ga:adClicks'].astype(int)

    if 'ga:impressions' in df.columns:
        df['ga:impressions']=df['ga:impressions'].apply(lambda x: int(x.replace(",","")))
        df['ga:impressions']=df['ga:impressions'].astype(int)
    
    if 'ga:transactions' in df.columns:
        df['ga:transactions']=df['ga:transactions'].apply(lambda x: int(x.replace(",","")))
        df['ga:transactions']=df['ga:transactions'].astype(int)
    
    if 'ga:transactionRevenue' in df.columns:
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].apply(lambda x: float(x.replace(",","")))
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].astype(float)

    
    return df


In [3]:
# Saatva.com
# date, sourceMedium      sessions, adClicks, impressions
ga_data_saatva=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_overall, end_date=End_Date_overall,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:date','ga:sourceMedium','ga:metro'])

ga_data_saatva['View']="Saatva.com"
ga_data_saatva['View_ID']="122377905"
ga_data_saatva.shape

(68403, 6)

In [4]:
# Saatvamattress.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_SM=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_overall, end_date=End_Date_overall,
                                            ga_metrics=['ga:sessions'],
                                            ga_dimensions=['ga:date','ga:sourceMedium','ga:metro'])

ga_data_SM['View']="Saatvamattress.com"
ga_data_SM['View_ID']="33605697"
ga_data_SM.shape

(145024, 6)

In [5]:
ga_data=ga_data_SM.append(ga_data_saatva)
ga_data=num_func_trans(ga_data)

In [6]:
ga_data['Source']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[0])
ga_data['Medium']=ga_data['ga:sourceMedium'].apply(lambda x: x.split(" / ")[1])

In [7]:
# msn in 'social'
# mail.yahoo.com / referral in 'referral'
# Social: 'msn','instagram','pinterest','facebook
                                                
ga_data['Defined_Media']=np.nan
ga_data['Defined_Media']=np.where(((ga_data['ga:sourceMedium'].str.contains('youtube.')) | (ga_data['Medium']=="social")),"Social",
                                  np.where(((ga_data['Source'].isin(['msn','instagram','Instagram','pinterest','facebook'])) & (ga_data['Medium']=='cpc')),"Social",
                                           np.where(ga_data['ga:sourceMedium'].str.contains('criteo'),"Retarget",
                                                    np.where(ga_data['Source']=="(direct)","Direct",
                                                             np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='cpc')),"SEM",
                                                                      np.where(((ga_data['Source'].isin(['google','bing','yahoo'])) & (ga_data['Medium']=='organic')),"SEO",
                                                                               np.where(ga_data['Source']=="powerInbox","Email",
                                                                                        np.where((ga_data['ga:sourceMedium'].str.contains('email')) & (~ga_data['ga:sourceMedium'].isin(["taboola / townnews-wvgazettemail","query.hemailaccessonline.com / referral"]) | (ga_data['ga:sourceMedium'].str.contains("mail.") )),"Email",
                                                                                                 np.where(ga_data['ga:sourceMedium'].str.contains('taboola'),"Video",
                                                                                                          np.where(ga_data['Medium']=="referral","Referral",
                                                                                                                   np.where(ga_data['Medium'].isin(["partner","partners"]),"partner",
                                                                                                                            np.where(ga_data['ga:sourceMedium'].str.contains('display'),"Display","Others")
                                                                                                                            )
                                                                                                                   )
                                                                                                          )
                                                                                                )
                                                                                       )
                                                                              )
                                                                     )
                                                            )
                                                   )
                                          )
                                 )
                                                         


In [8]:
GA_taxonomy=ga_data[['ga:sourceMedium','Source','Medium','Defined_Media']].drop_duplicates()
GA_taxonomy.reset_index(inplace=True)
GA_taxonomy=GA_taxonomy.sort_values('Defined_Media')
del GA_taxonomy['index']

In [9]:
GA_taxonomy.shape

(11141, 4)

In [10]:
ga_data.head(2)

,ga:date,ga:metro,ga:sessions,ga:sourceMedium,View,View_ID,Source,Medium,Defined_Media
0,20180901,(not set),147,(direct) / (none),Saatvamattress.com,33605697,(direct),(none),Direct
1,20180901,Albany GA,8,(direct) / (none),Saatvamattress.com,33605697,(direct),(none),Direct


In [11]:
ga_agg_session=pd.DataFrame()
ga_agg_wide_session=pd.DataFrame()
for metro,group in ga_data.groupby(['ga:metro']):
    df=group.groupby(['ga:date','Defined_Media'])['ga:sessions'].sum().to_frame()
    df.reset_index(inplace=True)
    df_wide=df.pivot('ga:date','Defined_Media','ga:sessions')
    df_wide.reset_index(inplace=True)
    df_wide['date']=df_wide['ga:date'].apply(lambda x:datetime.datetime.strptime(x,"%Y%m%d"))
    df['ga:metro']=metro
    df_wide['ga:metro']=metro
    ga_agg_session=ga_agg_session.append(df)
    ga_agg_wide_session=ga_agg_wide_session.append(df_wide)

ga_agg_wide_session=ga_agg_wide_session.fillna(0)
ga_agg_wide_session=ga_agg_wide_session[['ga:date','ga:metro']+[x for x in ga_agg_wide_session.columns.tolist() if x not in ['date','ga:date','ga:metro']]]
ga_agg_wide_session.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_overall+'_to_'+End_Date_overall+week+"_"+"Session"+'.csv',index=False)


In [12]:
writer=pd.ExcelWriter(writer_folder+'/Saatva_S_and_SM_GA_by_Media_'+Start_Date_overall+'_to_'+End_Date_overall+week+'.xlsx', 
                      engine='xlsxwriter',options={'strings_to_urls': False})

In [13]:
# Combined both Saatva.com and Saatvamattress.com
ga_agg_wide_session.to_excel(writer,"Session_hour_media",index=False)

ga_data.to_excel(writer,"data",index=False)
GA_taxonomy.to_excel(writer,"taxonomy",index=False)
writer.save()
GA_taxonomy.to_csv(writer_folder+'/Saatva_S_and_SM_GA_taxonomy_'+Start_Date_overall+'_to_'+End_Date_overall+week+'.csv',index=False)

In [14]:
ga_agg_session_by_View=ga_data.groupby(['View','View_ID','ga:date','ga:metro','Defined_Media'])['ga:sessions'].sum().unstack('Defined_Media')
ga_agg_session_by_View.reset_index(inplace=True)
ga_agg_session_by_View=ga_agg_session_by_View.fillna(0)
ga_agg_session_by_View.to_csv(writer_folder+'/Saatva_S_and_SM_GA_by_Media_by_View'+Start_Date_overall+'_to_'+End_Date_overall+week+"_Sessions"+'.csv',index=False)




# Google AdWords

In [15]:
def get_client(config_path='googleads_saatva.yaml'):
    return adwords.AdWordsClient.LoadFromStorage(config_path)

def download_report_as_stream(client, report_type, fields, start_date, end_date):
    '''Download Adwords report, then save as csv file to base_dir
    Args:
        client: adwords.AdWordsClient.LoadFromStorage()
        report_type: str eg. 'KEYWORDS_PERFORMANCE_REPORT'
        fields: list eg. ["CampaignId", "CampaignName", ... ]
        start_date: str, eg. '2018-02-11'
        end_date: str, eg.
    Return:
        str, path to csv file
    '''
    report_downloader = client.GetReportDownloader(version='v201802')
    # Create report definition.
    report = {
        'reportName': report_type,
        'dateRangeType': 'CUSTOM_DATE',
        'reportType': report_type,
        'downloadFormat': 'CSV',
        'selector': {
            'fields': fields,
            'dateRange': {
                'min': start_date,
                'max': end_date}
        }
    }

    # save output
    return report_downloader.DownloadReportAsStream(report, 
        skip_report_header=True, skip_column_header=False,
        skip_report_summary=True)



In [16]:
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","AdvertisingChannelType"]
#1
client_Saatva = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva.yaml')
response_Saatva = download_report_as_stream(client_Saatva, report_type, fields, Start_Date_overall, End_Date_overall)
Saatva_Click = pd.read_csv(response_Saatva).drop_duplicates()
#2
client_Saatva_Branding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Branding.yaml')
response_Saatva_Branding = download_report_as_stream(client_Saatva_Branding, report_type, fields, Start_Date_overall, End_Date_overall)
Saatva_Brand_Click = pd.read_csv(response_Saatva_Branding).drop_duplicates()
#3
client_Saatva_Competitors = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Competitors.yaml')
response_Saatva_Competitor = download_report_as_stream(client_Saatva_Competitors, report_type, fields, Start_Date_overall, End_Date_overall)
Saatva_Competitor_Click = pd.read_csv(response_Saatva_Competitor).drop_duplicates()
#4
client_TheSaatvaCompany = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompany.yaml')
response_TheSaatvaCompany = download_report_as_stream(client_TheSaatvaCompany, report_type, fields, Start_Date_overall, End_Date_overall)
TheSaatvaCompany_Click = pd.read_csv(response_TheSaatvaCompany).drop_duplicates()
#5
client_TheSaatvaCompanyBranding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompanyBranding.yaml')
response_TheSaatvaCompanyBranding = download_report_as_stream(client_TheSaatvaCompanyBranding, report_type, fields, Start_Date_overall, End_Date_overall)
TheSaatvaCompanyBranding_Click = pd.read_csv(response_TheSaatvaCompanyBranding).drop_duplicates()

#
Campign_Name_Type=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click).append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)

print(Campign_Name_Type.shape)
print(Campign_Name_Type[['Campaign','Advertising Channel']].drop_duplicates().shape)
print(len(Campign_Name_Type['Campaign'].unique()))
# Make sure unique Campaign match unique Advertising Channel

Campign_Name_Type.head(2)

(1573, 3)
(1424, 2)
1424


,Account,Campaign,Advertising Channel
0,Saatva,[ADL] [Non-Brand] [US] Mattresses - Back (Broa...,Search
1,Saatva,[ADL] [Non-Brand] [US] Mattresses - Back (Exac...,Search


In [17]:
client_Saatva = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva.yaml')
client_Saatva_Branding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Branding.yaml')
client_Saatva_Competitors = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Competitors.yaml')
client_TheSaatvaCompany = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompany.yaml')
client_TheSaatvaCompanyBranding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompanyBranding.yaml')


In [18]:
fields = ["AccountDescriptiveName","CampaignName","Date","CountryCriteriaId","MetroCriteriaId","Impressions", "Clicks", "Cost"]
report_type = 'GEO_PERFORMANCE_REPORT'

def getting_geo_df(client_yaml, report_type, fields, Start_Date_overall, End_Date_overall):
    response_Saatva = download_report_as_stream(client_yaml, report_type, fields, Start_Date_overall, End_Date_overall)
    df = pd.read_csv(response_Saatva)
    return df

Geo_Saatva = getting_geo_df(client_Saatva, report_type, fields, Start_Date_overall, End_Date_overall)
Geo_Saatva['Cost']=Geo_Saatva['Cost'].apply(lambda x: x/1000000)

Geo_Saatva_Branding = getting_geo_df(client_Saatva_Branding, report_type, fields, Start_Date_overall, End_Date_overall)
Geo_Saatva_Branding['Cost']=Geo_Saatva_Branding['Cost'].apply(lambda x: x/1000000)

Geo_Saatva_Competitors = getting_geo_df(client_Saatva_Competitors, report_type, fields, Start_Date_overall, End_Date_overall)
Geo_Saatva_Competitors['Cost']=Geo_Saatva_Competitors['Cost'].apply(lambda x: x/1000000)

Geo_TheSaatvaCompany = getting_geo_df(client_TheSaatvaCompany, report_type, fields, Start_Date_overall, End_Date_overall)
Geo_TheSaatvaCompany['Cost']=Geo_TheSaatvaCompany['Cost'].apply(lambda x: x/1000000)

Geo_TheSaatvaCompanyBranding = getting_geo_df(client_TheSaatvaCompanyBranding, report_type, fields, Start_Date_overall, End_Date_overall)
Geo_TheSaatvaCompanyBranding['Cost']=Geo_TheSaatvaCompanyBranding['Cost'].apply(lambda x: x/1000000)




/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2787: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [19]:
Geo_All_5=Geo_Saatva.append(Geo_Saatva_Branding).append(Geo_Saatva_Competitors).append(Geo_TheSaatvaCompany).append(Geo_TheSaatvaCompanyBranding)

Metro_id_list=Geo_All_5['Metro area'].unique().tolist()
Metro_id_list.remove(" --")
Metro_id_list=Metro_id_list+[int(x) for x in Metro_id_list]

Country_id_list=Geo_All_5['Country/Territory'].unique().tolist()
Country_id_list.remove(" --")
Country_id_list=Country_id_list+[int(x) for x in Country_id_list]

In [23]:
DMA_ID_Name=pd.DataFrame()
count_i=0
for client_ in [client_Saatva,client_Saatva_Branding,client_Saatva_Competitors,client_TheSaatvaCompany,client_TheSaatvaCompanyBranding]:

    location_criterion_service = client_.GetService('LocationCriterionService', version='v201802')

    location_id_list = Metro_id_list

    selector = {
      'fields': ['Id', 'LocationName', 'DisplayType', 'CanonicalName',
                 'ParentLocations', 'Reach', 'TargetingStatus'],
      'predicates': [{
          'field': 'Id',
          'operator': 'IN',
          'values': location_id_list
      }]
    }
    response_list = location_criterion_service.get(selector)
    for response in response_list:
        DMA_id=response['location']['id']
        DMA_type=response['location']['displayType']
        DMA_name=response['location']['locationName']

        df=pd.DataFrame({"DMA_id":DMA_id,"DMA_type":DMA_type,"DMA_name":DMA_name},index=[count_i])
        count_i+=1
        DMA_ID_Name=DMA_ID_Name.append(df)
DMA_ID_Name=DMA_ID_Name.drop_duplicates()

In [24]:
Country_ID_Name=pd.DataFrame()
count_i=0
for client_ in [client_Saatva,client_Saatva_Branding,client_Saatva_Competitors,client_TheSaatvaCompany,client_TheSaatvaCompanyBranding]:

    location_criterion_service = client_.GetService('LocationCriterionService', version='v201802')

    location_id_list = Country_id_list

    selector = {
      'fields': ['Id', 'LocationName', 'DisplayType', 'CanonicalName',
                 'ParentLocations', 'Reach', 'TargetingStatus'],
      'predicates': [{
          'field': 'Id',
          'operator': 'IN',
          'values': location_id_list
      }]
    }
    response_list = location_criterion_service.get(selector)
    for response in response_list:
        country_id=response['location']['id']
        country_type=response['location']['displayType']
        country_name=response['location']['locationName']

        df=pd.DataFrame({"Country_id":country_id,"Country_type":country_type,"Country_name":country_name},index=[count_i])
        count_i+=1
        Country_ID_Name=Country_ID_Name.append(df)
Country_ID_Name=Country_ID_Name.drop_duplicates()



In [25]:
DMA_ID_Name.to_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/DMA_Name_ID.csv",index=False)
DMA_id_list=[200602,200501,200623,200613,200618,200511]
DMA_id_list=DMA_id_list+[str(x) for x in DMA_id_list]
Geo_All_5_US=Geo_All_5[Geo_All_5['Country/Territory'].isin(['2840',2840])] # 2840 US
Geo_All_5_US_6_DMAs=Geo_All_5_US[Geo_All_5_US['Metro area'].isin(DMA_id_list)]

DMA_ID_Name=DMA_ID_Name.rename(columns={"DMA_id":"Metro area",'DMA_type':"displayType"})
DMA_ID_Name['Metro area']=DMA_ID_Name['Metro area'].astype(str)
Geo_All_5_US_6_DMAs=pd.merge(Geo_All_5_US_6_DMAs,DMA_ID_Name,on="Metro area",how="left")


In [26]:
'''
Saatva_Click=Saatva_Click[~((Saatva_Click['Campaign'].str.contains("Loom")) | (Saatva_Click['Campaign'].str.contains("Leaf")) | (Saatva_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompany_Click=TheSaatvaCompany_Click[~((TheSaatvaCompany_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompanyBranding_Click=TheSaatvaCompanyBranding_Click[~((TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Zenhaven")))]
'''
# No need to exclude the compaigns because of the small pctg


Geo_Saatva_Competitors=Geo_Saatva_Competitors[Geo_Saatva_Competitors['Campaign'].apply(lambda x: x[len(x)-4:len(x)+1])=="[SM]"]
Geo_Saatva_Competitors.reset_index(inplace=True)
del Geo_Saatva_Competitors['index']
# Keep only campaign labeled as "[SM]" in Saatva Competitor account

Geo_Saatva=pd.merge(Geo_Saatva,Campign_Name_Type,on=['Account','Campaign'],how="left")
Geo_Saatva_Branding=pd.merge(Geo_Saatva_Branding,Campign_Name_Type,on=['Account','Campaign'],how="left")
Geo_Saatva_Competitors=pd.merge(Geo_Saatva_Competitors,Campign_Name_Type,on=['Account','Campaign'],how="left")
Geo_TheSaatvaCompany=pd.merge(Geo_TheSaatvaCompany,Campign_Name_Type,on=['Account','Campaign'],how="left")
Geo_TheSaatvaCompanyBranding=pd.merge(Geo_TheSaatvaCompanyBranding,Campign_Name_Type,on=['Account','Campaign'],how="left")

Geo_Saatva['Campaign_Type']=Geo_Saatva['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
Geo_Saatva_Branding['Campaign_Type']=Geo_Saatva_Branding['Advertising Channel'].apply(lambda x: x.replace("Search","SEM_Brand").replace("Shopping","PLA"))
Geo_Saatva_Competitors['Campaign_Type']="SEM_Non_Brand" # Because it's all labeled as Non-Brand in Campaign and in Search Only Campaign Type
Geo_TheSaatvaCompany['Campaign_Type']=Geo_TheSaatvaCompany['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
Geo_TheSaatvaCompanyBranding['Campaign_Type']=Geo_TheSaatvaCompanyBranding['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))


In [27]:
# Split Saatva
# 'Search Network with Display Select' Campaign Type all 0 Impr and Clicks
Geo_Saatva_Search=Geo_Saatva[Geo_Saatva['Advertising Channel']=='Search']
Geo_Saatva_Others=Geo_Saatva[Geo_Saatva['Advertising Channel']!='Search']

Geo_Saatva_Search['Campaign_Type']=np.where((Geo_Saatva_Search['Campaign'].str.contains("Branded")) | (Geo_Saatva_Search['Campaign'].str.contains(" Brand ")) | (Geo_Saatva_Search['Campaign'].str.contains("Co-Brand")) | (Geo_Saatva_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((Geo_Saatva_Search['Campaign'].str.contains("Non-Brand")) | (Geo_Saatva_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
Geo_Saatva_Search_1=Geo_Saatva_Search[Geo_Saatva_Search['Campaign_Type']!="NA"]
Geo_Saatva_Search_2=Geo_Saatva_Search[Geo_Saatva_Search['Campaign_Type']=="NA"]
len(Geo_Saatva_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [28]:
# Split TheSaatvaCompany

Geo_TheSaatvaCompany_Search=Geo_TheSaatvaCompany[Geo_TheSaatvaCompany['Advertising Channel']=='Search']
Geo_TheSaatvaCompany_Others=Geo_TheSaatvaCompany[Geo_TheSaatvaCompany['Advertising Channel']!='Search']

Geo_TheSaatvaCompany_Search['Campaign_Type']=np.where((Geo_TheSaatvaCompany_Search['Campaign'].str.contains("Branded")) | (Geo_TheSaatvaCompany_Search['Campaign'].str.contains(" Brand ")) | (Geo_TheSaatvaCompany_Search['Campaign'].str.contains("Co-Brand")) | (Geo_TheSaatvaCompany_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((Geo_TheSaatvaCompany_Search['Campaign'].str.contains("Non-Brand")) | (Geo_TheSaatvaCompany_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
Geo_TheSaatvaCompany_Search_1=Geo_TheSaatvaCompany_Search[Geo_TheSaatvaCompany_Search['Campaign_Type']!="NA"]
Geo_TheSaatvaCompany_Search_2=Geo_TheSaatvaCompany_Search[Geo_TheSaatvaCompany_Search['Campaign_Type']=="NA"]
len(Geo_TheSaatvaCompany_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [29]:
Geo_Saatva_Search_2.head(2)

,Account,Campaign,Day,Country/Territory,Metro area,Impressions,Clicks,Cost,Advertising Channel,Campaign_Type


In [30]:
Geo_Saatva=Geo_Saatva_Search.append(Geo_Saatva_Others)
Geo_TheSaatvaCompany=Geo_TheSaatvaCompany_Search.append(Geo_TheSaatvaCompany_Others)



In [31]:
writer=pd.ExcelWriter(writer_folder+"/Saatva_AdWords_by_DMA_"+Start_Date_overall+"_to_"+End_Date_overall+week+"_cost.xlsx",engine='xlsxwriter')

In [32]:
Saatva_Agg=Geo_Saatva.append(Geo_Saatva_Branding).append(Geo_Saatva_Competitors).append(Geo_TheSaatvaCompany).append(Geo_TheSaatvaCompanyBranding)
Saatva_Agg=pd.merge(Saatva_Agg,DMA_ID_Name,on=['Metro area'],how="left")

Saatva_Agg['Day']=Saatva_Agg['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Saatva_Agg.reset_index(inplace=True)
del Saatva_Agg['index']
Saatva_Agg_Category=Saatva_Agg[['Campaign','Account','Advertising Channel','Campaign_Type']].drop_duplicates()
Saatva_Agg_Category.to_excel(writer,"Campaign Category",index=False)

In [33]:
Saatva_Agg['Day'][0]

datetime.date(2018, 10, 12)

In [34]:
date_list=["2018-11-29","2018-11-30","2018-12-01","2018-12-02","2018-12-03","2018-12-04","2018-12-05"]
date_list=[datetime.datetime.strptime(x,"%Y-%m-%d").date() for x in date_list]

Saatva_Agg['DMA_name']=Saatva_Agg['DMA_name'].fillna(" --")
Saatva_Agg['Country/Territory']=Saatva_Agg['Country/Territory'].fillna(" --")
Saatva_Agg['displayType']=Saatva_Agg['displayType'].fillna(" --")
Saatva_Agg['Campaign_Type']=Saatva_Agg['Campaign_Type'].fillna(" --")
Saatva_Agg[(Saatva_Agg['Day'].isin(date_list)) & (Saatva_Agg['Campaign_Type']=="SEM_Brand")]['Clicks'].sum()

8279

In [35]:
Saatva_Agg['displayType'].unique()

array(['DMA Region', ' --'], dtype=object)

In [36]:
Saatva_Agg['Campaign_Type'].unique()

array(['SEM_Non_Brand', 'SEM_Brand', 'Display', 'Video', 'PLA'], dtype=object)

In [37]:
Saatva_Agg=Saatva_Agg.groupby(["Campaign_Type","DMA_name","displayType","Country/Territory","Day"])['Impressions','Clicks', 'Cost'].sum().reset_index()

Saatva_Agg['CTR']=Saatva_Agg['Clicks']/Saatva_Agg['Impressions']
Saatva_Agg.to_excel(writer,"reshaped by dma day",index=False)

Saatva_Agg_No_NA=Saatva_Agg[(Saatva_Agg['Campaign_Type']!="NA") & (Saatva_Agg['Campaign_Type']!="Search Network with Display Select")]

Saatva_Agg_No_NA.to_excel(writer,"clean data by dma day",index=False)

In [38]:
Saatva_Agg[(Saatva_Agg['Day'].isin(date_list)) & (Saatva_Agg['Campaign_Type']=="SEM_Brand")]['Clicks'].sum()

8279

In [39]:
writer.save()

In [40]:
AdWords_output=Saatva_Agg.copy()
AdWords_output['Date']=AdWords_output['Day'].apply(lambda x: str(x)[0:4]+str(x)[5:7]+str(x)[8:10])
AdWords_output=AdWords_output[['Campaign_Type',"DMA_name","displayType","Country/Territory",'Date','Impressions','Clicks','Cost']]
AdWords_output=AdWords_output.rename(columns={"Impressions":"IMP"})
AdWords_output=AdWords_output.sort_values(['Campaign_Type',"DMA_name","displayType","Country/Territory",'Date'],ascending=[True,True,True,True,True])
AdWords_output.to_csv(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Date_overall+week+"_cost.csv",index=False)

In [41]:
Saatva_Agg_Category.to_csv(writer_folder+"/Saatva_AdWords_Taxonomy_"+Start_Date_overall+"_to_"+End_Date_overall+week+"_cost.csv",index=False)


In [42]:
date_list_str=["20181129","20181130","20181201","20181202","20181203","20181204","20181205"]

AdWords_output[(AdWords_output['Date'].isin(date_list_str)) & (AdWords_output['Campaign_Type']=="SEM_Brand")]['Clicks'].sum()


8279

In [43]:
AdWords_output.head(2)

,Campaign_Type,DMA_name,displayType,Country/Territory,Date,IMP,Clicks,Cost
0,Display,--,--,2012,20180902,13,0,0.0
1,Display,--,--,2016,20180904,1,0,0.0
